In [12]:
import paddle
import numpy as np
import pandas as pd
import math 
import sys
import sklearn
import paddle.fluid as fluid
train_data = np.arange(0,5).astype('float32').reshape([-1,1])
y_true = np.arange(1,6).astype('float32').reshape([-1,1])


In [19]:
x = fluid.layers.data(name='x', shape=[1], dtype='float32')
y = fluid.layers.data(name='y', shape=[1], dtype='float32')
y_predict = fluid.layers.fc(input=x, size=1, act=None)
cost = fluid.layers.square_error_cost(input=y_predict, label=y)
avg_cost = fluid.layers.mean(cost)

sgd_optimizer = fluid.optimizer.Adamax(learning_rate=0.1)
sgd_optimizer.minimize(avg_cost)
use_cuda = False
cpu = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program())
for i in range(100):
    outs = exe.run(
    feed = {'x':train_data, 'y':y_true},
    fetch_list = [y_predict, avg_cost])
#     print(outs[1])
print(outs)

[array([[0.9323227],
       [1.9577475],
       [2.9831724],
       [4.008597 ],
       [5.034022 ]], dtype=float32), array([0.00157601], dtype=float32)]


In [1]:
import paddle
import numpy as np
import pandas as pd
import math 
import sys
import sklearn
import paddle.fluid as fluid

In [2]:
train_data = pd.read_csv("housing.data", sep='\s+', header=None)
train_data = train_data.iloc[:, :-1]
train_label = train_data.iloc[:, -1]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(train_data, train_label)
y_train = pd.DataFrame(y_train)

In [6]:
# X_train = pd.DataFrame(np.arange(0,10))
# X_test = pd.DataFrame(np.arange(1,11))
x_1 = fluid.layers.data(name = 'x', shape=[13])
y_1 = fluid.layers.data(name = 'y', shape=[1])
y_predict = fluid.layers.fc(input=x_1, size=1, act = None)

cost = fluid.layers.square_error_cost(input=y_predict, label=y_1)
avg_cost = fluid.layers.mean(cost)

sgd_optimizer = fluid.optimizer.SGD(learning_rate=0.01)
sgd_optimizer.minimize(avg_cost)


cpu = fluid.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program())
for i in range(100):
    outs = exe.run(
        feed = {'x' : X_train, 'y': y_train},
        fetch_list = [y_predict, avg_cost]
    )
# print(outs)

ValueError: Variable x has been created before. the previous shape is (-1, 1); the new shape is (-1, 13). They are not matched.

In [1]:
import sys
import math
import numpy as np
import paddle

import paddle.fluid as fluid

In [12]:
x = fluid.layers.data(name='x', shape=[13])
y = fluid.layers.data(name='y', shape=[1])
y_predict = fluid.layers.fc(input=x, size=1, act=None)
cost = fluid.layers.square_error_cost(input=y_predict, label=y)
avg_cost = fluid.layers.mean(cost)

sgd_optimizer = fluid.optimizer.SGD(learning_rate=0.01)
sgd_optimizer.minimize(avg_cost)

train_reader = paddle.batch(
    paddle.reader.shuffle(paddle.dataset.uci_housing.train(), buf_size=500), batch_size = 20
)
place = fluid.CPUPlace()

feeder = fluid.DataFeeder(place=place, feed_list=[x, y])
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())
PASS_NUM = 100
for pass_id in range(PASS_NUM):
    for data in train_reader():
        avg_loss_value = exe.run(fluid.default_main_program(),
                                feed = feeder.feed(data),
                                 fetch_list = [y_predict, avg_cost]
                                )
#         print(avg_loss_value[1])

        


[[18.212729]
 [23.085712]
 [20.656569]
 [32.40638 ]
 [17.155891]
 [25.254904]
 [26.685865]
 [26.63507 ]
 [24.988728]
 [16.036724]
 [26.688023]
 [25.852917]
 [24.718922]
 [23.57695 ]
 [20.1408  ]
 [27.985806]
 [31.045666]
 [19.38115 ]
 [21.15141 ]
 [17.400356]]
[[29.370415 ]
 [25.897867 ]
 [18.870382 ]
 [15.9395485]
 [23.217737 ]
 [30.791607 ]
 [23.739876 ]
 [23.475712 ]
 [28.345116 ]
 [24.209717 ]
 [22.399597 ]
 [21.073982 ]
 [19.554613 ]
 [20.940933 ]
 [ 8.4314   ]
 [15.392666 ]
 [26.06752  ]
 [31.191826 ]
 [21.99711  ]
 [30.972466 ]]
[[25.488907]
 [18.39233 ]
 [21.771584]
 [24.957853]
 [24.139473]
 [26.257906]
 [19.682047]
 [24.81009 ]
 [11.315512]
 [30.63302 ]
 [22.437809]
 [39.94317 ]
 [35.330242]
 [38.85831 ]
 [14.979326]
 [12.537592]
 [31.532438]
 [20.71108 ]
 [34.09763 ]
 [22.04275 ]]
[[30.854052]
 [22.297604]
 [36.044193]
 [21.586231]
 [25.134634]
 [23.48627 ]
 [24.768564]
 [23.766428]
 [23.29063 ]
 [21.039238]
 [36.12359 ]
 [29.8953  ]
 [ 9.601608]
 [24.427868]
 [24.36289 ]
 [

In [8]:
batch_size = 20
train_reader = paddle.batch(
    paddle.reader.shuffle(paddle.dataset.uci_housing.train(), buf_size=500),
    batch_size = batch_size
)
test_reader = paddle.batch(
    paddle.reader.shuffle(paddle.dataset.uci_housing.test(), buf_size=500),
    batch_size = batch_size
)

x = fluid.layers.data(name='x', shape=[13])
y = fluid.layers.data(name='y', shape=[1])
y_predict = fluid.layers.fc(input=x, size=1, act=None)

main_program = fluid.default_main_program()
startup_program = fluid.default_startup_program()

cost = fluid.layers.square_error_cost(input=y_predict, label=y)
avg_cost = fluid.layers.mean(cost)

sgd_optimizer = fluid.optimizer.SGD(learning_rate=0.01)
sgd_optimizer.minimize(avg_cost)

place = fluid.CPUPlace()
exe = fluid.Executor(place)

feeder = fluid.DataFeeder(place=place, feed_list=[x, y])
num_epoch = 100
exe.run(startup_program)

for i in range(num_epoch):
    for data in train_reader():
        outer = exe.run(main_program,
            feed = feeder.feed(data),
            fetch_list = [avg_cost]
        )
    outer

In [ ]:
train_prompt = "Train cost"
test_prompt = "Test cost"
step = 0
exe_test = fluid.Executor(place)